In [4]:
import os, argparse
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ["https://www.googleapis.com/auth/drive.file"]

def creds():
    c = None
    if os.path.exists("token.json"):
        c = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not c or not c.valid:
        if c and c.expired and c.refresh_token:
            c.refresh(Request())
        else:
            try:
                flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
                c = flow.run_local_server(port=0)
            except Exception:
                flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
                c = flow.run_console()
        with open("token.json", "w") as f:
            f.write(c.to_json())
    return c

def upload(file_path, folder_id, name=None):
    service = build("drive", "v3", credentials=creds())
    meta = {"name": name or os.path.basename(file_path), "parents": [folder_id]}
    media = MediaFileUpload(file_path, resumable=True)
    req = service.files().create(body=meta, media_body=media, fields="id,name,size")
    resp = None
    while resp is None:
        status, resp = req.next_chunk()
        if status:
            print(f"Uploaded {int(status.progress()*100)}%", end="\r")
    print(f"\nDone: {resp['name']} (id={resp['id']})")



In [5]:
if __name__ == "__main__":
    upload("data_textfeat.zip", "COMP4222_data_and_checkpoints", "data_textfeat.zip")

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'